In [3]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.integrate import solve_ivp


# Network parameters
n = 10  # Number of nodes
m = 2   # Number of edges to attach from a new node to existing nodes
G = nx.barabasi_albert_graph(n, m)

# Time span for the simulation
t_span = (0, 10)
t_eval = np.linspace(*t_span, 10)

# Parameters
beta = 0.1
gamma = 0.3

# Initial conditions
S0 = np.ones(n) - 0.1
I0 = np.zeros(n); 
I0[0] = 0.1  
R0 = np.zeros(n)

# Flatten initial conditions for ODE solver
y0 = np.concatenate([S0, I0, R0])

# The system of differential equations
def SIRCON_ENSEMBLE(t, y, G, beta, gamma):
    # Unpack the current state for each node
    S, I, R = np.split(y, 3)
    
    dS_dt = np.zeros(n)
    dI_dt = np.zeros(n)
    dR_dt = np.zeros(n)
    
    for node in G.nodes:
        neighbors = list(G.neighbors(node))
        infection_rate = 1 - np.prod([1 - beta * I[neighbor] for neighbor in neighbors])
        
        dS_dt[node] = -S[node] * infection_rate
        dI_dt[node] = S[node] * infection_rate - I[node] * gamma
        dR_dt[node] = I[node] * gamma
    
    return np.concatenate([dS_dt, dI_dt, dR_dt])



# Solve the system of differential equations
# solution = solve_ivp(derivatives, t_span, y0, args=(G, beta, gamma), t_eval=t_eval, method='RK45')

# # Reshape and plot the results for a selected node
# num_nodes = G.number_of_nodes()
# S, I, R = np.split(solution.y, 3)

# plt.figure(figsize=(14, 6))

# # Example: plotting for node 0
# node_index = 0
# plt.plot(solution.t, S[node_index], label="Susceptible")
# plt.plot(solution.t, I[node_index], label="Infected")
# plt.plot(solution.t, R[node_index], label="Recovered")
# plt.xlabel("Time (days)")
# plt.ylabel("Population")
# plt.legend()
# plt.title(f"Dynamics of SIR Model for Node {node_index} in Barabási-Albert Network")
# plt.show()
